In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings, pickle, os
warnings.filterwarnings("ignore")

boundary = 1
gap = 40
init = 19
min_change = -20

In [2]:
input_ = {
    "model_id": "20230807120000000",
}

In [3]:
root = r"C:\Users\tzuli\Documents\python\teco"

# 取得predict位置
pred_path = os.path.join(root, "data", "predict")        
os.makedirs(pred_path, exist_ok = True)
output_json = os.path.join(pred_path, "output.json")

# 取得model位置     
model_detail = os.path.join(root, "data", "train", input_["model_id"], "model")

# 取得train位置
train_path = os.path.join(root, "data", "train")
data_csv = os.path.join(train_path, "train_data.csv")

In [4]:
df0 = pd.read_csv(data_csv)
print(df0.shape)
df0.head(3)

(278, 16)


,日期,工號,序號,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,平衡_F側角度,平衡_F側配重,最終_L側角度,最終_L側不平衡量,最終_F側角度,最終_F側不平衡量
0,2023-06-01,FC236606T1-2,1,590,12.0,266.0,259.0,181.0,12.0,275.0,259.0,180.0,358.0,58.20,293.0,23.00
1,2023-06-01,FC236606T1-2,2,590,358.0,58.2,293.0,23.0,358.0,60.0,293.0,23.0,307.0,3.50,241.0,3.90
2,2023-06-01,FC236606T1-2,3,590,307.0,3.5,241.0,3.9,307.0,4.0,241.0,4.0,322.0,0.46,214.0,0.36


In [5]:
df0 = df0.query("日期 >= '2023-07-07'")
df0.shape

(48, 16)

### 模型平均調整次數

In [6]:
side = "L"
features = pickle.load(open(os.path.join(model_detail, f"feat_order.pkl"), "rb"))
l_outlier_boundary = pickle.load(open(os.path.join(model_detail, f"{side}_outlier_boundary.pkl"), "rb"))
l_skew_feat = pickle.load(open(os.path.join(model_detail, f"{side}_skew_feat.pkl"), "rb"))
l_pt = pickle.load(open(os.path.join(model_detail, f"{side}_power_tf.pkl"), "rb"))
l_scaler = pickle.load(open(os.path.join(model_detail, f"{side}_scaler.pkl"), "rb"))
l_model = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

side = "F"
f_outlier_boundary = pickle.load(open(os.path.join(model_detail, f"{side}_outlier_boundary.pkl"), "rb"))
f_skew_feat = pickle.load(open(os.path.join(model_detail, f"{side}_skew_feat.pkl"), "rb"))
f_pt = pickle.load(open(os.path.join(model_detail, f"{side}_power_tf.pkl"), "rb"))
f_scaler = pickle.load(open(os.path.join(model_detail, f"{side}_scaler.pkl"), "rb"))
f_model = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

df = df0.query("序號 == 1")

In [7]:
df = df0.dropna().reset_index(drop = True)
df = df.query("(初始_L側不平衡量 < @init) and (初始_F側不平衡量 < @init)")
df = df.drop_duplicates(subset = "工號", keep = "first")

In [8]:
df_input = df.loc[:, :"初始_F側不平衡量"]
df_input.columns = ['日期', '工號', '序號', "speed", "l_angle_ori", "l_weight_ori", "f_angle_ori", "f_weight_ori"]
inputs = df_input.to_dict(orient = "records")

In [9]:
result_df = pd.DataFrame()
for input_ in tqdm(inputs):
    date = input_["日期"]
    order = input_["工號"]
    op = int(input_["序號"])
    speed = int(input_["speed"])
    l_angle_ori = int(input_["l_angle_ori"])
    l_weight_ori = round(float(input_["l_weight_ori"]))
    f_angle_ori = int(input_["f_angle_ori"])
    f_weight_ori = round(float(input_["f_weight_ori"]))



    l_weight_change, f_weight_change = [], []
    for i in range(gap+1):
        l_weight_change.extend([max(min_change, l_weight_ori + i), max(min_change, l_weight_ori - i)])
        f_weight_change.extend([max(min_change, f_weight_ori + i), max(min_change, f_weight_ori - i)])
    l_weight_change = list(set(l_weight_change))
    f_weight_change = list(set(f_weight_change))


    l_angle_change0, f_angle_change0 = [], []
    for i in range(0, gap+1, 10):
        l_angle_change0.extend([(l_angle_ori + i), (l_angle_ori - i)])
        f_angle_change0.extend([(f_angle_ori + i), (f_angle_ori - i)])
    l_angle_change0 = list(set(l_angle_change0))
    f_angle_change0 = list(set(f_angle_change0))



    l_angle_change = []
    for angle in l_angle_change0:
        if angle > 360:
            l_angle_change.append(angle - 360)
        elif angle < 0:
            l_angle_change.append(360 - angle)
        else:
            l_angle_change.append(angle)


    f_angle_change = []
    for angle in f_angle_change0:
        if angle > 360:
            f_angle_change.append(angle - 360)
        elif angle < 0:
            f_angle_change.append(360 - angle)
        else:
            f_angle_change.append(angle)
    

    all_combinations = []

    # 嵌套循环生成所有组合
    for a in l_angle_change :
        for b in l_weight_change:
            for c in f_angle_change :
                for d in f_weight_change:
                    x = [date, order, op, speed, l_angle_ori, l_weight_ori, f_angle_ori, f_weight_ori, a, b, c, d]
                    all_combinations.append(x)



    df_comb = pd.DataFrame(
        all_combinations, 
        columns = ['日期', '工號', '序號', '平衡轉速', '初始_L側角度', '初始_L側不平衡量', '初始_F側角度',
                '初始_F側不平衡量', '平衡_L側角度', '平衡_L側配重', '平衡_F側角度', '平衡_F側配重']
        )
    df_pred = df_comb.copy()



    df_pred["預期_F側不平衡量"] = df_pred["初始_F側不平衡量"] - df_pred["平衡_F側配重"]
    df_pred["預期_L側不平衡量"] = df_pred["初始_L側不平衡量"] - df_pred["平衡_L側配重"]

    df_pred["初始_不平衡量_diff"] = abs(df_pred["初始_L側不平衡量"] - df_pred["初始_F側不平衡量"])
    df_pred["平衡_不平衡量_diff"] = abs(df_pred["平衡_L側配重"] - df_pred["平衡_F側配重"])

    diff = abs(df_pred["初始_L側角度"] - df_pred["初始_F側角度"])
    df_pred["初始_角度_diff"] = diff.apply(lambda X: min(X, 360 - X))
    diff = abs(df_pred["平衡_L側角度"] - df_pred["平衡_F側角度"])
    df_pred["平衡_角度_diff"] = diff.apply(lambda X: min(X, 360-X))



    l_df, f_df = df_pred.copy(), df_pred.copy()



    for col in features[1:]:    
        max_ = l_outlier_boundary[col]["max"]
        max_index = l_df.index[l_df[col] > max_].to_list()
        l_df.loc[max_index, col] = max_

        min_ = l_outlier_boundary[col]["min"]
        min_index = l_df.index[l_df[col] < min_].to_list()
        l_df.loc[min_index, col] = min_

        max_ = f_outlier_boundary[col]["max"]
        max_index = f_df.index[f_df[col] > max_].to_list()
        f_df.loc[max_index, col] = max_

        min_ = f_outlier_boundary[col]["min"]
        min_index = f_df.index[f_df[col] < min_].to_list()
        f_df.loc[min_index, col] = min_


    
    l_df[l_skew_feat] = l_pt.transform(l_df[l_skew_feat])
    f_df[f_skew_feat] = f_pt.transform(f_df[f_skew_feat])
    l_df[features] = l_scaler.transform(l_df[features])
    f_df[features] = f_scaler.transform(f_df[features])



    pred_l = l_model.predict(l_df.loc[:, "平衡轉速":])
    pred_f = f_model.predict(f_df.loc[:, "平衡轉速":])



    df_comb["最終_L側不平衡量"] = pred_l
    df_comb["最終_F側不平衡量"] = pred_f



    # df_comb["總不平衡量"] = df_comb.eval("(最終_L側不平衡量 + 最終_F側不平衡量)")
    df_comb["總不平衡量"] = df_comb.eval("(abs(最終_L側不平衡量) + abs(最終_F側不平衡量))")
    
    # 先嚴格篩選，在寬鬆篩選
    df_comb1 = df_comb.query("((平衡_L側配重 > 0) and (平衡_F側配重 > 0)) and ((最終_L側不平衡量 > 0) and (最終_F側不平衡量 > 0))")
    if len(df_comb1) == 0:
        df_comb1 = df_comb.query("(最終_L側不平衡量 > 0) and (最終_F側不平衡量 > 0)")
    if len(df_comb1) == 0:
        df_comb1 = df_comb.copy()
    comb_index = df_comb1["總不平衡量"].idxmin()



    result_df = pd.concat([result_df, df_comb.loc[[comb_index]]], ignore_index = True)

100%|██████████| 9/9 [00:22<00:00,  2.46s/it]


In [10]:
result_df[["日期", "工號", "初始_L側不平衡量", "初始_F側不平衡量", "最終_L側不平衡量", "最終_F側不平衡量"]].round(2)

,日期,工號,初始_L側不平衡量,初始_F側不平衡量,最終_L側不平衡量,最終_F側不平衡量
0,2023-07-07,FC237402-1,11,11,0.55,0.00
1,2023-07-07,FC237402-2,8,18,0.02,0.01
2,2023-07-07,FC237402-3,1,1,0.22,0.11
3,2023-07-10,FC237524,9,15,0.49,0.61
4,2023-07-10,FC237002-1,12,12,0.01,0.00
5,2023-07-10,FC237002-2,5,5,0.15,0.70
6,2023-07-10,FC233525-1,4,6,0.21,0.00
7,2023-07-10,FC237603,7,17,0.32,0.83
8,2023-07-11,FC237523-1,6,5,0.87,0.01


### 人工平均調整次數

In [11]:
df_human = df0.query("序號 != 1")
df_human = df_human.query("工號 in @result_df['工號']")
df_g = df_human.groupby(["日期", "工號"]).agg({"序號": "count"})
df_g

序號
日期         工號            
2023-07-07 FC237402-1   4
           FC237402-2   3
           FC237402-3   3
2023-07-10 FC233525-1   2
           FC237002-1   3
           FC237002-2   4
           FC237524     8
           FC237603     2
2023-07-11 FC237523-1   4

In [12]:
mean_count = df_g["序號"].sum()/len(df_g)
print(f"工號數: {len(df_g)}, 平均工號調整次數: {round(mean_count, 2)}")

工號數: 9, 平均工號調整次數: 3.67


In [ ]:
# 